### Single Chain Task
See below a function calling example which generates the output in a single chain. see the `run` method. It has four components/[runnables](https://python.langchain.com/v0.1/docs/expression_language/interface/) all in one chain.

In [ ]:
class PhoneStatusAgent:
    def __init__(self):
        self.llm = Ollama(model="llama3", format="json")
        self.tools = [phone_status_checker, phone_reactivator]
        rendered_tools = render_text_description(self.tools)
        system_prompt = f"""You are an assistant that has access to the following set of tools.
            Here are the names and descriptions for each tool:

            {rendered_tools}
            Given the user input, return the name and input of the tool to use.
            Return your response as a JSON blob with 'name' and 'arguments' keys.
            The value associated with the 'arguments' key should be a dictionary of parameters."""
        self.prompt = ChatPromptTemplate.from_messages(
            [("system", system_prompt), ("user", "{input}")]
        )
        self.chat_history = []

    def run(self, input_text):
        question = input_text
        chain = self.prompt | self.llm | JsonOutputParser() | self.tool_chain
        response = chain.invoke({"input": question})
        return response

    def tool_chain(self, model_output):
        tool_map = {tool.name: tool for tool in self.tools}
        chosen_tool = tool_map[model_output["name"]]
        return itemgetter("arguments") | chosen_tool

### Runnable/Chain Components
The main chain in the above example consists of four components:

`self.prompt | self.llm | JsonOutputParser() | self.tool_chain`

However, you can modify it according to your use case. For instance, if you need the output of `JsonOutputParser()` for something else before running the final chain `self.tool_chain`, you can split the chain into two parts: one with the first three components and the other with the last one.

Let’s say we want to produce the final answer using the LLM, and for that, we need the generated function-calling information _(output of `JsonOutputParser()`)_ to generate the correct response. We can update our main PhoneStatusAgent class as shown below.

In [ ]:
class PhoneNumberStatusAgent:
    def __init__(self):
        self.llm = Ollama(model="llama3", format="json")
        self.tools = [phone_number_status_checker, phone_number_reactivator]
        rendered_tools = render_text_description(self.tools)

        tool_system_prompt = f"""You are an assistant that has access to the following set of tools.
            Here are the names and descriptions for each tool:

            {rendered_tools}
            Given the user input, return the name and input of the tool to use.
            Return your response as a JSON blob with 'name' and 'arguments' keys.
            The value associated with the 'arguments' key should be a dictionary of parameters."""
        self.tool_prompt = ChatPromptTemplate.from_messages(
            [("system", tool_system_prompt), ("user", "{input}")]
        )

        response_template = """You are a helpful assistant who generates a generic message based on the output of a tool execution.

        Tool Output Information:
        {tool_execution}

        Response Instructions:
        - If phone number status is inactive, ask, "Do you want to reactivate the phone number?"
        - Remove any information related to the tool.
        - Return your response as a JSON object with a 'message' key.
        """
        self.response_prompt = ChatPromptTemplate.from_template(response_template)

    def run(self, input_text):
        question = input_text

				# Chain one with three components
        tool_chain = self.tool_prompt | self.llm | JsonOutputParser()
        tool_chain_output = tool_chain.invoke({"input": question})

        tool_name = tool_chain_output["name"]
        tool_arguments = tool_chain_output["arguments"]
				
				# Chain two with only one component which is basichally to invoke the actual tool/function
        tool_output = self.get_tool(tool_name).invoke({"input": "", **tool_arguments})

				# Chain three to generate the final response
        response_chain = self.response_prompt | self.llm | JsonOutputParser()
        response = response_chain.invoke(
            {"tool_execution": {"tool_name": tool_name, "tool_output": tool_output}}
        )

        return response

    def get_tool(self, tool_name):
        tool_map = {tool.name: tool for tool in self.tools}
        chosen_tool = tool_map[tool_name]
        return chosen_tool